In [1]:
import cv2
import random
import numpy as np
import matplotlib.pyplot as plt
import polars as pl
from pathlib import Path
from utils import init_dataframe, Pipeline
import segmentation
import logging
logging.basicConfig(filename='log.txt',
                    format='%(asctime)s,%(msecs)03d-%(name)s-%(levelname)s: %(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S',
                    level=logging.DEBUG)

logger = logging.getLogger(__name__)


In [2]:
# Set random seed for reproducibility
RANDOM_SEED = 42
random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)

# Set dataset path
TRAIN_DATA = Path("data/ISIC2018_Task3_Training_Input")
TRAIN_LABELS = Path("data/ISIC2018_Task3_Training_GroundTruth/ISIC2018_Task3_Training_GroundTruth.csv")

VALID_DATA = Path("data/ISIC2018_Task3_Validation_Input")
VALID_LABELS = Path("data/ISIC2018_Task3_Validation_GroundTruth/ISIC2018_Task3_Validation_GroundTruth.csv")

TEST_DATA = Path("data/ISIC2018_Task3_Test_Input")
TEST_LABELS = Path("data/ISIC2018_Task3_Test_GroundTruth/ISIC2018_Task3_Test_GroundTruth.csv")

PROCESSED_IMAGES_PATH = Path("data/processed/")


In [3]:
train_df = init_dataframe(TRAIN_DATA, TRAIN_LABELS)
valid_df = init_dataframe(VALID_DATA, VALID_LABELS)
test_df = init_dataframe(TEST_DATA, TEST_LABELS)
pipeline = Pipeline(train_df=train_df, valid_df=valid_df, test_df=test_df, output_path=PROCESSED_IMAGES_PATH)

pipeline.print_df()


Train DataFrame shape: (10015, 10)
Columns: ['sample_id', 'image', 'image_path', 'MEL', 'NV', 'BCC', 'AKIEC', 'BKL', 'DF', 'VASC']
shape: (5, 10)
┌───────────┬──────────────┬─────────────────────────────────┬─────┬───┬───────┬─────┬─────┬──────┐
│ sample_id ┆ image        ┆ image_path                      ┆ MEL ┆ … ┆ AKIEC ┆ BKL ┆ DF  ┆ VASC │
│ ---       ┆ ---          ┆ ---                             ┆ --- ┆   ┆ ---   ┆ --- ┆ --- ┆ ---  │
│ i64       ┆ str          ┆ str                             ┆ f64 ┆   ┆ f64   ┆ f64 ┆ f64 ┆ f64  │
╞═══════════╪══════════════╪═════════════════════════════════╪═════╪═══╪═══════╪═════╪═════╪══════╡
│ 0         ┆ ISIC_0024306 ┆ data\ISIC2018_Task3_Training_I… ┆ 0.0 ┆ … ┆ 0.0   ┆ 0.0 ┆ 0.0 ┆ 0.0  │
│ 1         ┆ ISIC_0024307 ┆ data\ISIC2018_Task3_Training_I… ┆ 0.0 ┆ … ┆ 0.0   ┆ 0.0 ┆ 0.0 ┆ 0.0  │
│ 2         ┆ ISIC_0024308 ┆ data\ISIC2018_Task3_Training_I… ┆ 0.0 ┆ … ┆ 0.0   ┆ 0.0 ┆ 0.0 ┆ 0.0  │
│ 3         ┆ ISIC_0024309 ┆ data\ISIC2018_Task3_Train

In [4]:

pipeline.apply_to_image_and_save('image_path', segmentation.apply_clahe, 'clahe', on_split=['test', 'valid', 'train'])

Column 'clahe' added: 11720 processed, 0 skipped (11720 total)


In [5]:
pipeline.apply_to_image_and_save('clahe', segmentation.apply_median_blur, 'median_blur', on_split=['test', 'valid', 'train'])

Column 'median_blur' added: 11720 processed, 0 skipped (11720 total)


In [6]:
pipeline.apply_to_image_and_save('median_blur', segmentation.apply_otsu_threshold, 'otsu', on_split=['test', 'valid', 'train'])

Column 'otsu' added: 11720 processed, 0 skipped (11720 total)


In [7]:
pipeline.apply_to_image_and_save('otsu', segmentation.apply_morph, 'morph', on_split=['test', 'valid', 'train'])

Column 'morph' added: 11720 processed, 0 skipped (11720 total)


In [8]:
pipeline.apply_to_image_and_save('morph', segmentation.find_max_contour, 'contour', on_split=['test', 'valid', 'train'])

Column 'contour' added: 11720 processed, 0 skipped (11720 total)


In [9]:
pipeline.print_df()

Train DataFrame shape: (10015, 15)
Columns: ['sample_id', 'image', 'image_path', 'MEL', 'NV', 'BCC', 'AKIEC', 'BKL', 'DF', 'VASC', 'clahe', 'median_blur', 'otsu', 'morph', 'contour']
shape: (5, 15)
┌───────────┬────────────┬────────────┬─────┬───┬────────────┬────────────┬────────────┬───────────┐
│ sample_id ┆ image      ┆ image_path ┆ MEL ┆ … ┆ median_blu ┆ otsu       ┆ morph      ┆ contour   │
│ ---       ┆ ---        ┆ ---        ┆ --- ┆   ┆ r          ┆ ---        ┆ ---        ┆ ---       │
│ i64       ┆ str        ┆ str        ┆ f64 ┆   ┆ ---        ┆ str        ┆ str        ┆ str       │
│           ┆            ┆            ┆     ┆   ┆ str        ┆            ┆            ┆           │
╞═══════════╪════════════╪════════════╪═════╪═══╪════════════╪════════════╪════════════╪═══════════╡
│ 0         ┆ ISIC_00243 ┆ data\ISIC2 ┆ 0.0 ┆ … ┆ data\proce ┆ data\proce ┆ data\proce ┆ data\proc │
│           ┆ 06         ┆ 018_Task3_ ┆     ┆   ┆ ssed\ISIC_ ┆ ssed\ISIC_ ┆ ssed\ISIC_ ┆ essed\